# Hybrid Filtering

**Hybrid filtering technique** is a combination of multiple recommendation techniques like, merging collaborative filtering (CF) with content-based filtering (CB) or vice-versa. Hybrid Filtering get two technique:
- Content-based → collaborative 
We can use content-based prediction at users with many training examples and collaboration at others. The prediction of content-based models can be used in recursive collaborative filtering.
- Collaborative → content-based 
Features can be extracted from other users’ ratings.

**Dataset**
- In this case, i'm using dataset that have been i create to collaborative and content_based filtering because i want to hybrid two dataset and get recomendations film for user that have been take rate film and get similarity from it.

**Case**
- On this dataset I use hybrid from content base and then collaborative filtering. So that the Workflownya is I look for similarity between movies then see the movie recommendation for users based on collaborative then new to the second, so that the film was obtained the recommendation for the user where the film has Similarity Movie (content_based) that has user rate on collaborative filtering.

## Analysis

In [27]:
#Import Libraries
from math import sqrt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity
%matplotlib inline

Import dataset

In [28]:
from recomendations_data import dataset

In [29]:
dfcontent=pd.read_csv('dataset film.csv')
dfrate=pd.DataFrame(dataset)

In [30]:
dfcontent

,Movie,Type,Genre,Rating,Duration
0,Ada Apa dengan Cinta 2,Indonesia,Romance,7.4,123
1,Gundala,Indonesia,Action,7.6,123
2,Dilan 1991,Indonesia,Drama,6.8,121
3,Bumi Manusia,Indonesia,Drama,7.2,181
4,Dua Garis Biru,Indonesia,Drama,8.2,113
5,Avenger: End Game,Hollywood,Action,8.6,181
6,The Lion King,Hollywood,Animation,7.1,118
7,Aladdin,Hollywood,Fantasy,7.1,128
8,Spiderman: Far From Home,Hollywood,Action,7.8,129
9,Captain Marvel,Hollywood,Action,7.0,123


In [31]:
dfrate

,ANI,AhokTemanFirli,Damar Teman Firli,Dpv,Febi ganteng gak ada obat,Genjeh,Hania,Indra 1991 SM,Indra Junior,Jawaharal,...,OM INDRA,Putrisqiana,Rima,Romantika,Star,Topik Zulkarnain,bunga,faizah,franadek,luck
Ada Apa dengan Cinta 2,4,0,5,5,4,5,3,0,4,2,...,3,4,5,5,4,0,0,3,4,3
Aladdin,4,0,0,0,5,5,0,0,5,5,...,1,0,5,0,5,0,5,0,5,0
Avengers: End Game,0,3,5,5,5,5,0,0,5,5,...,5,5,5,0,5,5,5,5,5,4
Bumi Manusia,5,0,0,0,0,0,4,0,0,0,...,0,4,4,0,0,0,0,5,5,0
Captain Marvel,4,4,0,5,4,4,0,0,5,4,...,5,3,5,0,5,2,5,0,4,2
Dilan 1991,4,0,0,4,4,3,4,0,0,3,...,2,2,5,5,0,0,4,5,4,0
Dua Garis Biru,0,0,0,0,0,0,4,0,4,5,...,5,3,3,0,0,0,0,4,3,0
Gundala,0,0,0,4,3,4,5,5,0,4,...,0,3,5,0,4,0,4,0,4,0
Spiderman: Far From Home,3,0,5,5,5,4,0,0,5,5,...,5,4,5,0,0,4,5,0,4,0
The Lion King,0,0,0,0,0,0,0,0,5,4,...,0,0,4,0,5,5,5,0,4,0


**Content Based Filtering**

In [32]:
# Definition Content Based Filtering
def content_based(person,min_content_score):
    
    #Set Index of dfrate
    k=0
    not_watch=[]
    for i in dfrate[person]:
        if i==0:
            not_watch.append(dfrate.index[k])
            k=k+1
        else:
            k=k+1
    
    #Set Index of dfcontent to counting cosine similarity
    dfcontent1=dfcontent.set_index('Movie')
    dfcontent1=pd.get_dummies(dfcontent1)
    
    #Cosine Similarity
    cosine_similarities = cosine_similarity(dfcontent1, dfcontent1)
    
    new=pd.DataFrame(cosine_similarities, index=dfcontent['Movie'],columns=dfcontent['Movie'])
    
    #Get Minimum content score for each user to recomendations film
    final=pd.DataFrame(new[not_watch].mean().sort_values(ascending=False),columns=['Score'])
    final2=final[final.Score>=min_content_score]
    indeks=final2.index
    
    return indeks

**Person Correlation**

In [33]:
def person_correlation(person1, person2):

   # To get both rated items
    both_rated = {}
    for item in dataset[person1]:
        if item in dataset[person2]:
            both_rated[item] = 1

    number_of_ratings = len(both_rated)

    # Checking for ratings in common
    if number_of_ratings == 0:
        return 0

    # Add up all the preferences of each user
    person1_preferences_sum = sum([dataset[person1][item] for item in both_rated])
    person2_preferences_sum = sum([dataset[person2][item] for item in both_rated])

    # Sum up the squares of preferences of each user
    person1_square_preferences_sum = sum([pow(dataset[person1][item],2) for item in both_rated])
    person2_square_preferences_sum = sum([pow(dataset[person2][item],2) for item in both_rated])

    # Sum up the product value of both preferences for each item
    product_sum_of_both_users = sum([dataset[person1][item] * dataset[person2][item] for item in both_rated])

    # Calculate the pearson score
    numerator_value = product_sum_of_both_users - (person1_preferences_sum*person2_preferences_sum/number_of_ratings)
    denominator_value = sqrt((person1_square_preferences_sum - pow(person1_preferences_sum,2)/number_of_ratings) * (person2_square_preferences_sum -pow(person2_preferences_sum,2)/number_of_ratings))

    if denominator_value == 0:
        return 0
    else:
        r = numerator_value / denominator_value
        return r

**Film_Recommendations**

In [34]:
#Get Score Rate and Film Recomendations from person        
def film_recommendations(person):

    # Gets recommendations for a person by using a weighted average of every other user's rankings
    totals = {}
    simSums = {}
    for other in dataset:
        # don't compare me to myself
        if other == person:
            continue
        sim = person_correlation(person,other)
        #print ">>>>>>>",sim

        # ignore scores of zero or lower
        if sim <=0: 
            continue
        for item in dataset[other]:

            # only score movies i haven't seen yet
            if item not in dataset[person] or dataset[person][item] == 0:

            # Similrity * score
                totals.setdefault(item,0)
                totals[item] += dataset[other][item]* sim
                # sum of similarities
                simSums.setdefault(item,0)
                simSums[item]+= sim

        # Create the normalized list

    rankings = [(total/simSums[item],item) for item,total in totals.items()]
    rankings.sort()
    rankings.reverse()
    return rankings

**Hybrid Filtering**

- Hybrid Filtering is combination of Content based and Collaborative Filtering.

In [35]:
def hybrid_filtering(person, min_content_score):
    content_based(person, min_content_score)
    return film_recommendations(person)

**Output**

In [36]:
hybrid_filtering('faizah', 0.1)

[(1.9919656843588625, 'Gundala'),
 (1.9280791056979967, 'Spiderman: Far From Home'),
 (1.6892548477063598, 'Captain Marvel'),
 (0.5236344746510952, 'Aladdin'),
 (0.22477984308407356, 'The Lion King')]

**Conclusion**

- Hybrid Filtering is a combination of content based and collaborative Filtering. The Output of Hybrid Filtering is the recommendation of a movie for users where the film recommendation is not only based on the rate given by other users but based on the similarities of the genre, duration and others. So it will be produced recommendation film in ranking based on score similarity. Recommended movies are movies that are not yet watched by the user.

- From the result of hybrid filtering to faizah with a minimum score of 0.1 it is seen that the film recommendation for Faizah is Gundala, Spiderman and so on. 

- For marketing analysis, these results can be used for customer segmentation more precisely in offering movies or promotional offers. So not everyone has the same prefrention towards the film and the precise segmentation will further improve effectiveness and profitability. 